# Web scraping - Extraccion de la data

Considerar el punto de que:
El web scraping que realizado, se encarga de descargar(link referenciando al archivo parket de la data) los archivos de la pagina web de TCL(Taxis & Limousine Commission), por lo que en realidad podrias decir cumplimos con una premisa del web scraping: la extracción de datos de una fuente externa, pero no realizamos una "disección" o "scraping"(en realidad referie a raspado-raspar) del documento html para luego procesar la data para construir y reorganizar los datos. En este caso, la data ya se obtiene en formato tabular.

In [ ]:
import requests
import os
import pandas as pd
import time
from selenium import webdriver # pip install selenium
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException


In [ ]:
# Crear una carpeta para las descargas si no existe
download_dir = os.path.join(os.getcwd(), "descargas")
if not os.path.exists(download_dir):
    os.makedirs(download_dir)
print(download_dir)

#### Web driver de Selenium

In [ ]:
# Configura el WebDriver (asegúrate de tener el driver correspondiente)
options = webdriver.ChromeOptions()
options.add_argument("--window-size=800,640")
#Directorio predeterminado para descargar los archivos
prefs = {
    "download.default_directory": download_dir,  # Cambiar a tu directorio de descargas
    "download.prompt_for_download": False,       # Desactivar el cuadro de diálogo de descarga
    "download.directory_upgrade": True,          # Permitir actualizar la carpeta de descarga
    "safebrowsing.enabled": True                 # Activar descarga segura
}
options.add_experimental_option("prefs", prefs)
service = Service(executable_path="chromedriver_win64\chromedriver.exe") #Ruta del chromedriver
options.headless = True  # Enable headless mode
#options.add_argument("--window-size=800,640")
driver = webdriver.Chrome(service=service, options=options)
driver.get('https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page') # URL pagina de la data

# Usar WebDriverWait para esperar hasta que la sección con la clase y el id estén visibles
wait = WebDriverWait(driver, 20)
faq_section_2023 = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.faq-answers#faq2023')))
faq_section_2024 = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.faq-answers#faq2024')))

# Buscar todos los enlaces dentro de la sección con class 'faq-answers' y id 'faq2023'
taxi_links_2023 = faq_section_2023.find_elements(By.CSS_SELECTOR, 'a[title="Yellow Taxi Trip Records"], a[title="Green Taxi Trip Records"]')
taxi_links_2024 = faq_section_2024.find_elements(By.CSS_SELECTOR, 'a[title="Yellow Taxi Trip Records"], a[title="Green Taxi Trip Records"]')

#### Descarga de archivos

In [ ]:
# Descargar cada archivo del año 2023
for url in taxi_links_2023:
    # Extraer el nombre del archivo de la URL
    url = url.get_attribute('href')
    file_name = url.split("/")[-1]
    file_path = os.path.join(download_dir, file_name)
    # Descargar el archivo
    print(f"Descargando {file_name} ...")
    #response = requests.get(url)
    try:
        response = requests.get(url, timeout=120)  # Aumentar el timeout a 120 segundos
    except requests.exceptions.Timeout:
        print(f"Error: El archivo {file_name} tardó demasiado en responder.")
        continue  # Salta al siguiente archivo si hay un problema de tiempo de espera
    # Verificar si la descarga fue exitosa
    if response.status_code == 200:
        # Guardar el archivo en la carpeta de descargas
        with open(file_path, 'wb') as f:
            f.write(response.content)
        print(f"Archivo descargado y guardado en: {file_path}")
    else:
        print(f"Error al descargar {file_name}: Código de estado {response.status_code}")
        

# Descargar cada archivo del año 2024
for url in taxi_links_2024:
    # Extraer el nombre del archivo de la URL
    url = url.get_attribute('href')
    file_name = url.split("/")[-1]
    file_path = os.path.join(download_dir, file_name)
    # Descargar el archivo
    print(f"Descargando {file_name} ...")
    #response = requests.get(url)
    try:
        response = requests.get(url, timeout=120)  # Aumentar el timeout a 120 segundos
    except requests.exceptions.Timeout:
        print(f"Error: El archivo {file_name} tardó demasiado en responder.")
        continue  # Salta al siguiente archivo si hay un problema de tiempo de espera
    # Verificar si la descarga fue exitosa
    if response.status_code == 200:
        # Guardar el archivo en la carpeta de descargas
        with open(file_path, 'wb') as f:
            f.write(response.content)
        print(f"Archivo descargado y guardado en: {file_path}")
    else:
        print(f"Error al descargar {file_name}: Código de estado {response.status_code}")

In [ ]:
# Cierra el driver
driver.quit()

#### Descarga de los datos acerca del gasto energetico

In [ ]:
# Crear una carpeta para las descargas si no existe
download_dir = os.path.join(os.getcwd(), "descargas\Fuel_rate")
if not os.path.exists(download_dir):
    os.makedirs(download_dir)
print(download_dir)
# Configura el WebDriver (asegúrate de tener el driver correspondiente)
options = webdriver.ChromeOptions()
options.add_argument("--window-size=800,640")
#Directorio predeterminado para descargar los archivos
prefs = {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,       
    "download.directory_upgrade": True,          
    "safebrowsing.enabled": True                 
}
options.add_experimental_option("prefs", prefs)
service = Service(executable_path="chromedriver_win64\chromedriver.exe") #Ruta del chromedriver
options.headless = True  # Enable headless mode
#options.add_argument("--window-size=800,640")
driver = webdriver.Chrome(service=service, options=options)
driver.get('https://open.canada.ca/data/en/dataset/98f1a129-f628-4ce4-b24d-6f16bf24dd64#wb-auto-6') # URL pagina de la data

try:
    wait = WebDriverWait(driver, 40)
    
    # Paso 1: Encontrar el contenedor que tiene los enlaces a las subpáginas
    main_section = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'section#dataset-resources.resources')))
    print("main_section",main_section)
    # Buscar los enlaces a las subpáginas (donde están los datasets)
    links = main_section.find_elements(By.CSS_SELECTOR, 'a.heading.resource-heading')
    print(links)
    # Limitar la iteración a los primeros 8 enlaces
    max_links = 8
    link_count = 0  # Contador para los enlaces procesados

    # Iterar sobre cada enlace a la subpágina
    for link in links:
        if link_count >= max_links:  # Rompe el loop si ya se procesaron los primeros 5 enlaces
            break

        href = link.get_attribute("href")  # Obtener el href de la subpágina
        print(f"Navegando a: {href}")
        driver.get(href)  # Navegar a la subpágina
        
        # Buscar los enlaces de los archivos en la subpágina
        try:
            # Esperar a que carguen los elementos en la nueva página
            subpage_wait = WebDriverWait(driver, 20)
            # Encontrar todos los enlaces a archivos CSV en la nueva página
            csv_links = subpage_wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a[href*="csv"]')))
            
            # Descargar cada archivo CSV encontrado
            for csv_link in csv_links:
                file_href = csv_link.get_attribute("href")
                file_name = file_href.split('/')[-1]  # Obtener el nombre del archivo
                file_path = os.path.join(download_dir, file_name)

                # Comprobar si el archivo ya existe
                if not os.path.exists(file_path):
                    print(f"Descargando archivo: {file_href}")
                    driver.get(file_href)  # Descargar el archivo automáticamente
                else:
                    print(f"El archivo {file_name} ya existe. Saltando descarga.")

        except TimeoutException:
            print("No se encontraron archivos CSV en la subpágina.")

        # Volver a la página anterior para continuar con los otros enlaces
        driver.back()

        link_count += 1  # Incrementar el contador después de procesar un enlace

except TimeoutException:
    print("No se encontraron los elementos dentro del tiempo de espera.")
    print(driver.page_source)  # Imprime el HTML para inspeccionar qué está cargando

finally:
    driver.quit()  # Cerrar el WebDriver al finalizar


# Cloud Test



### Descarga del primer archivo

In [23]:
# Crear una carpeta para las descargas si no existe
download_dir = os.path.join(os.getcwd(), "descargas")
if not os.path.exists(download_dir):
    os.makedirs(download_dir)
print(download_dir)


options = webdriver.ChromeOptions()
options.add_argument("--window-size=800,640")
#options.add_argument("--headless")
options.add_argument('--headless')  # Activa el modo headless
#options.add_argument('--disable-gpu')  # Desactiva la GPU para una ejecución más estable en algunos sistemas
prefs = {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
}
options.add_experimental_option("prefs", prefs)
service = Service(executable_path="chromedriver_win64/chromedriver.exe")  # Ruta del chromedriver
#options.headless = True  # Ejecutar en modo headless

driver = webdriver.Chrome(service=service, options=options)
driver.get('https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page')  # URL de la página de datos

# Esperar hasta que la sección de enlaces de 2023 esté visible
wait = WebDriverWait(driver, 20)
faq_section_2023 = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.faq-answers#faq2023')))

# Encontrar todos los enlaces dentro de la sección
taxi_links_2023 = faq_section_2023.find_elements(By.CSS_SELECTOR, 'a[title="Green Taxi Trip Records"]')

# Extraer la URL del primer elemento
first_link = taxi_links_2023[0].get_attribute('href')

# Extraer el nombre del archivo de la URL
file_name = first_link.split("/")[-1]
file_path = os.path.join(download_dir, file_name)

# Descargar el archivo
print(f"Descargando {file_name} ...")
try:
    response = requests.get(first_link, timeout=120)  # Aumentar el timeout a 120 segundos
except requests.exceptions.Timeout:
    print(f"Error: El archivo {file_name} tardó demasiado en responder.")
    driver.quit()  # Cierra el navegador si hay un problema de tiempo de espera
    raise  # Lanza la excepción para que el script falle si es necesario

# Verificar si la descarga fue exitosa
if response.status_code == 200:
    with open(file_path, 'wb') as f:
        f.write(response.content)
    print(f"Archivo descargado y guardado en: {file_path}")
else:
    print(f"Error al descargar {file_name}: Código de estado {response.status_code}")

# Cerrar el navegador al final
driver.quit()

c:\Pablo\datasets\TaxisNY\descargas
Descargando green_tripdata_2023-01.parquet ...
Archivo descargado y guardado en: c:\Pablo\datasets\TaxisNY\descargas\green_tripdata_2023-01.parquet


### Incremental Load


In [24]:
# Create a folder for downloads if it doesn't exist
download_dir = os.path.join(os.getcwd(), "descargas")  # Define the download directory
if not os.path.exists(download_dir):  # Check if the directory exists
    os.makedirs(download_dir)  # Create the directory if it doesn't exist
print(download_dir)

# List to store the downloaded months (as integers)
meses_descargados = []

# Iterate over the files in the directory
for archivo in os.listdir(download_dir):
    # Check if the file is not a folder
    if os.path.isfile(os.path.join(download_dir, archivo)):
        try:
            # Extract the month (assuming the format is something like 'archivo_01-2023.csv')
            fragmento = archivo.split('.')[0][-2:]  # Get the last two characters before the file extension
            meses_descargados.append(fragmento)  # Add the month to the list
        except Exception as e:
            print(f"Error processing the file {archivo}: {e}")

print("Previously downloaded months:", meses_descargados)

# Selenium WebDriver setup
options = webdriver.ChromeOptions()
options.add_argument("--window-size=800,640")  # Set the browser window size
options.add_argument('--headless')
prefs = {
    "download.default_directory": download_dir,  # Set the download directory
    "download.prompt_for_download": False,  # Disable download prompt
    "download.directory_upgrade": True,  # Allow upgrading the directory for downloads
    "safebrowsing.enabled": True  # Enable safe browsing
}
options.add_experimental_option("prefs", prefs)  # Add preferences to the WebDriver options
service = Service(executable_path="chromedriver_win64/chromedriver.exe")  # Path to chromedriver executable
driver = webdriver.Chrome(service=service, options=options)  # Initialize WebDriver
driver.get('https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page')  # Open the webpage

# Wait until the section with 2023 links is visible
wait = WebDriverWait(driver, 20)
faq_section_2023 = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.faq-answers#faq2023')))
taxi_links_2023 = faq_section_2023.find_elements(By.CSS_SELECTOR, 'a[title="Green Taxi Trip Records"]')

# Download files for months that have not been downloaded
for mes in meses_descargados:
    try:
        mes_num = int(mes)  # Convert the month to an integer
        mes_siguiente = mes_num + 1  # Calculate the next month
        mes_siguiente_str = str(mes_siguiente).zfill(2)  # Format the next month as a 2-digit string
        
        # Check if the file for the next month already exists
        file_name = f"green_tripdata_2023-{mes_siguiente_str}.parquet"  # Define the expected file name
        file_path = os.path.join(download_dir, file_name)
        if os.path.exists(file_path):  # If the file exists, skip the download
            print(f"File for month {mes_siguiente_str} already downloaded, skipping download.")
            continue
        
        # Only continue if the index is valid
        indice = mes_siguiente - 1
        if indice < len(taxi_links_2023):  # Ensure the index is within the range of available links
            enlace = taxi_links_2023[indice].get_attribute('href')  # Get the download link for the month
            print(f"Downloading file for month {mes_siguiente_str} from {enlace}")

            try:
                # Make an HTTP request to download the file
                response = requests.get(enlace, timeout=120)  # Set a timeout for the request
                if response.status_code == 200:  # Check if the request was successful
                    with open(file_path, 'wb') as f:
                        f.write(response.content)  # Write the downloaded content to the file
                    print(f"File downloaded and saved to: {file_path}")
                else:
                    print(f"Error downloading {file_name}: Status code {response.status_code}")
            except requests.exceptions.Timeout:
                print(f"Error: File {file_name} took too long to respond.")
                driver.quit()  # Quit the WebDriver if there's a timeout
                raise  # Re-raise the exception to stop further execution

        else:
            print("No more months to download. Process finished.")
            break  # Exit the loop if there are no more months to download

    except Exception as e:
        print(f"Error processing month {mes}: {e}")  # Catch any other errors during processing

# Close the WebDriver when done
driver.quit()


c:\Pablo\datasets\TaxisNY\descargas
Previously downloaded months: ['01']
File downloaded and saved to: c:\Pablo\datasets\TaxisNY\descargas\green_tripdata_2023-02.parquet
